# 課題 Excel業務の自動化のアイデアを考えてみよう

商品レビューをポジティブ/ネガティヴに分類する。

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
from openai.types.chat import ChatCompletionToolParam

# モデル名
MODEL_NAME = "gpt-4o-mini"
# 入力ファイル名
INPUT_FILE = "商品レビュー.xlsx"
# 出力ファイル名
OUTPUT_FILE = "商品レビュー分析結果.xlsx"

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# レビュー感情分析用のツール定義
tools = [
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "analyze_review_sentiment",
            "description": "商品レビューの感情をポジティブまたはネガティブに分類する",
            "parameters": {
                "type": "object",
                "properties": {
                    "review_analysis": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "review": {
                                    "type": "string",
                                    "description": "元のレビューテキスト"
                                },
                                "sentiment": {
                                    "type": "string",
                                    "description": "感情分析結果（ポジティブまたはネガティブ）",
                                    "enum": ["ポジティブ", "ネガティブ"]
                                }
                            },
                            "required": ["review", "sentiment"]
                        }
                    }
                },
                "required": ["review_analysis"]
            }
        }
    })
]

print("レビュー感情分析を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel(INPUT_FILE, sheet_name='レビュー')

# 2. データをLLM用にテキスト形式に変換
reviews = df.iloc[:, 0].tolist()  # A列のデータをリストとして取得
reviews_text = "\n".join([f"{i+1}. {review}" for i, review in enumerate(reviews)])

prompt_text = f"""
以下の商品レビューを分析し、それぞれをポジティブまたはネガティブに分類してください。
各レビューについて、必ず以下の2つの情報を含めてください：
1. review: 元のレビューテキスト
2. sentiment: 感情分析結果（「ポジティブ」または「ネガティブ」）

レビュー一覧：
{reviews_text}
"""
print(prompt_text)

# 3. OpenAI APIの呼び出し
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "あなたは商品レビューの感情分析を行う専門家です。各レビューを読んで、ポジティブまたはネガティブに分類してください。"},
        {"role": "user", "content": prompt_text}
    ],
    tools=tools,
    tool_choice={
        "type": "function",
        "function": {"name": "analyze_review_sentiment"}
    }
)

# 4. 分析結果をJSONオブジェクトに変換
# ツール呼出情報を取得
tool = response.choices[0].message.tool_calls[0]
# JSONオブジェクトに変換
data = json.loads(tool.function.arguments)

# データフレームに変換
df_out = pd.DataFrame(data['review_analysis'])

# 5. 結果をExcelファイルに保存
df_out.to_excel(OUTPUT_FILE, index=False)

print(f"{OUTPUT_FILE} ファイルに分析結果を保存しました。")

レビュー感情分析を開始します。

以下の商品レビューを分析し、それぞれをポジティブまたはネガティブに分類してください。
各レビューについて、必ず以下の2つの情報を含めてください：
1. review: 元のレビューテキスト
2. sentiment: 感情分析結果（「ポジティブ」または「ネガティブ」）

レビュー一覧：
1. めちゃくちゃ使いやすい！
2. おいしかったです
3. 商品写真とまったく違うものが届いた
4. しばらく使っているうちにネジが緩んできて壊れてしまった
5. 可もなく不可もなくといったところ

商品レビュー分析結果.xlsx ファイルに分析結果を保存しました。
